In [1]:
import os
import re
from collections import defaultdict

import pandas as pd

import camelot

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [2]:
dir_item = '/home/jake/Documents/News_Item/'
dir_proj = dir_item + 'fukushima/'
dir_tableau = dir_proj + 'tableau_data/'

dir_pdf = './pdf/'
dir_tables = './tables/'
dir_table_pages = [dir_tables + d for d in sorted(os.listdir(dir_tables)) if not d.startswith('.')]

files = []

for dir_page in dir_table_pages:
    tmp_files = [dir_page + '/' + f for f in os.listdir(dir_page)]
    files.extend(tmp_files)

In [3]:
dir_table_pages
print(len(files))
files[:2]

12490


['./tables/001/일본산+및+기타국가산+수입수산물+방사능검사+현황(8월+21일+완료건)-page-2-table-1.csv',
 './tables/001/일본산+수입축산물+방사능검사+현황(2019.8.21)-page-1-table-1.csv']

In [4]:
only_names = [f.split('/')[3] for f in files]
print(len(only_names))
only_names[:2]

12490


['일본산+및+기타국가산+수입수산물+방사능검사+현황(8월+21일+완료건)-page-2-table-1.csv',
 '일본산+수입축산물+방사능검사+현황(2019.8.21)-page-1-table-1.csv']

In [5]:
dedup = set(files)
len(dedup)

12490

편의상 파일 이름만 사용하려고 했는데 (regex 만들 때 편함) 예상치 못한 오류 발견  
2015년 파일과 2016년 파일의 이름이 같을 수 있다  
그러므로 page까지 포함해야 함

In [6]:
from collections import Counter
cnt = Counter(only_names)
# [c for c in cnt if c[1]!=1]
cnt.most_common(10)

[('일본산+및+기타국가산+수입수산물+방사능검사+현황(2월7일+완료건)-page-1-table-1.csv', 2),
 ('일본산+및+기타국가산+수입수산물+방사능검사+현황(2월12일+완료건)-page-2-table-1.csv', 2),
 ('일본산+및+기타국가산+수입수산물+방사능검사+현황(2월8일+완료건)-page-2-table-1.csv', 2),
 ('일본산+및+기타국가산+수입수산물+방사능검사+현황(2월12일+완료건)-page-1-table-1.csv', 2),
 ('일본산+및+기타국가산+수입수산물+방사능검사+현황(2월13일+완료건)-page-2-table-1.csv', 2),
 ('일본산+및+기타국가산+수입수산물+방사능검사+현황(2월8일+완료건)-page-1-table-1.csv', 2),
 ('일본산+및+기타국가산+수입수산물+방사능검사+현황(2월13일+완료건)-page-1-table-1.csv', 2),
 ('일본산+및+기타국가산+수입수산물+방사능검사+현황(2월1일+완료건)-page-3-table-1.csv', 2),
 ('일본산+및+기타국가산+수입수산물+방사능검사+현황(1월29일+완료건)-page-2-table-1.csv', 2),
 ('일본산+및+기타국가산+수입수산물+방사능검사+현황(2월1일+완료건)-page-1-table-1.csv', 2)]

In [7]:
[f for f in files if '일본산+및+기타국가산+수입수산물+방사능검사+현황(2월7일+완료건)-page-1-table-1.csv' in f]

['./tables/045/일본산+및+기타국가산+수입수산물+방사능검사+현황(2월7일+완료건)-page-1-table-1.csv',
 './tables/126/일본산+및+기타국가산+수입수산물+방사능검사+현황(2월7일+완료건)-page-1-table-1.csv']

---

# 파일 이름 분류하기  

**Hypothesis**  
- 수입식품: 숫자로 시작하는 파일, '수입식품'을 포함한 파일  
- 축수산물: 수산물, 축산물, 축수산물 포함  
- 일본산과 기타국가는 어떻게 구분?

## 수입식품: imported

In [8]:
import re

In [9]:
imported0 = [f for f in files if re.match(r'^\.\/tables\/\d{3}\/\d{8}[+-_]방사능.*', f)]
len(imported0)

808

In [10]:
imported1 = [f for f in files if re.match(r'^\.\/tables\/\d{3}\/\d{8}[-]page.*', f)]
len(imported1)

2690

In [11]:
imported2 = [f for f in files if '수입식품' in f]
len(imported2)

3361

In [12]:
imported3 = [f for f in files if re.match(r'^\.\/tables\/\d{3}\/\d{8}[+]일일보고.*', f)]
len(imported3)

186

In [13]:
imported4 = [f for f in files if re.match(r'^\.\/tables\/\d{3}\/\d{8}[-]1.*', f)]
len(imported4)

6

In [14]:
imported = []
imported.extend(imported0 + imported1 + imported2 + imported3 + imported4)
len(imported)

7051

---

## 축수산물: fish_and_meat

In [15]:
fish_and_meat0 = [f for f in files if '축수산물' in f]
len(fish_and_meat0)

1901

In [16]:
fish = [f for f in files if ('수산물' in f) and ('축수산물' not in f)]
len(fish)

2409

In [17]:
meat = [f for f in files if '축산물' in f]
len(meat)

769

In [18]:
fish_and_meat = []
fish_and_meat.extend(fish_and_meat0 + fish + meat)
len(fish_and_meat)

5079

축수산물은 처음부터 통합되어 관리되다가 나중에 분리됐기 때문에 파일로 분류하기 어렵다  
게다가 일본과 기타국가도 섞여있는 경우가 많아서 파일로 분리하기 어렵다  

1. 우선 전부 합친다
2. 축수산물은 분류하지 않는다
3. 일본은 일본 도시명으로 분류한다

---

## 기타국가 일반식품: foreign

In [19]:
categorized = imported + fish_and_meat
len(categorized)

12130

전체 파일 중 수입식품과 축수산물로 분류된 파일 수

In [20]:
not_cat = [f for f in files if f not in categorized]
len(not_cat)

360

분류되지 않은 파일 수

In [21]:
not_cat[-5:]

['./tables/357/20141219+기타국가+방사능+검출+현황-page-1-table-1.csv',
 './tables/363/20141119+기타국가+방사능+검출+현황(프랑스산+기타잼류)-page-1-table-1.csv',
 './tables/381/20140822+기타국가+방사능+검출+현황(러시아산+차가버섯)-page-1-table-1.csv',
 './tables/397/20140609+기타국가+방사능+검출+현황-page-1-table-1.csv',
 './tables/401/20140519+기타국가+방사능+검출+현황-page-1-table-1.csv']

In [22]:
not_cat_foreign = [f for f in not_cat if '기타' in f]
len(not_cat_foreign)

360

분류되지 않은 모든 파일에 '기타' 키워드가 포함되어 있다.  
처음부터 끝까지 같은 형식의 파일로 보임.

In [23]:
not_cat[200:205]

['./tables/122/20180223_기타국가_방사능_검출_현황-page-1-table-1.csv',
 './tables/123/20180220_기타국가_방사능_검출_현황-page-1-table-1.csv',
 './tables/123/20180221_기타국가_방사능_검출_현황-page-1-table-1.csv',
 './tables/124/20180212_기타국가_방사능_검출_현황-page-1-table-1.csv',
 './tables/124/20180214_기타국가_방사능_검출_현황-page-1-table-1.csv']

In [24]:
not_cat[300:305]

['./tables/220/20161010+기타국가+방사능+검출+현황(중국+능이,+링곤베리잼)-page-1-table-1.csv',
 './tables/223/20160923+기타국가+방사능+검출+현황(중국능이)-page-1-table-1.csv',
 './tables/223/20160927+기타국가+방사능+검출+현황(중국+냉동능이+2건)-page-1-table-1.csv',
 './tables/223/20160928+기타국가+방사능+검출+현황(중국+건조능이+2건)-page-1-table-1.csv',
 './tables/223/20160928+기타국가+방사능+검출+현황(터키+밀크초코릿헤이즐넛)-page-1-table-1.csv']

In [25]:
not_cat[-5:]

['./tables/357/20141219+기타국가+방사능+검출+현황-page-1-table-1.csv',
 './tables/363/20141119+기타국가+방사능+검출+현황(프랑스산+기타잼류)-page-1-table-1.csv',
 './tables/381/20140822+기타국가+방사능+검출+현황(러시아산+차가버섯)-page-1-table-1.csv',
 './tables/397/20140609+기타국가+방사능+검출+현황-page-1-table-1.csv',
 './tables/401/20140519+기타국가+방사능+검출+현황-page-1-table-1.csv']

not_cat: 기타국가 방사능 검출 현황(기타국가 일반식품)  

In [26]:
foreign = not_cat
len(foreign)

360

---

파일 로드, 테이블 추출  
컬럼 수가 테이블 종류를 결정한다고 가정한다

In [27]:
from collections import defaultdict

def get_df_from_files(files):
    col_2_df = defaultdict(pd.DataFrame)

    for f in files:
        tmp = pd.read_csv(f)
        tmp['file'] = f
        col_len = len(tmp.columns)
        tmp.columns = list(range(col_len))
        col_2_df[col_len] = col_2_df[col_len].append(tmp, sort=False)
    
    return col_2_df

In [28]:
col_2_df_imported = get_df_from_files(imported)

In [29]:
col_2_df_fish_meat = get_df_from_files(fish_and_meat)

In [30]:
col_2_df_foreign = get_df_from_files(foreign)

csv 파일을 읽는데 시간이 너무 오래 걸린다  
중간 저장을 해야겠다

In [31]:
col_2_df_imported.keys()

dict_keys([9, 12, 7, 6, 2])

In [32]:
col_2_df_fish_meat.keys()

dict_keys([9, 7, 4])

In [33]:
col_2_df_foreign.keys()

dict_keys([9, 7, 6])

In [34]:
for k,v in col_2_df_imported.items():
    print(f'{k} columns, {v.shape[0]} rows')

9 columns, 147541 rows
12 columns, 9111 rows
7 columns, 215 rows
6 columns, 3 rows
2 columns, 15 rows


12 컬럼: 필요없는 데이터, 9컬럼과 종류가 다름  
7,6,2 컬럼: 9컬럼의 오류, 하지만 9컬럼에 비해 숫자가 적기 때문에 무시해도 될듯?  
체크사항: 세슘, 요오드가 검출됐기 때문에 (숫자값) 오류가 발생했다면 무시할 수 없다

In [35]:
for k,v in col_2_df_fish_meat.items():
    print(f'{k} columns, {v.shape[0]} rows')

9 columns, 53525 rows
7 columns, 5 rows
4 columns, 3981 rows


4 컬럼: 종류가 다른 필요없는 데이터

In [36]:
col_2_df_fish_meat[4].head()

,0,1,2,3
0,NaN,계,육류 : 10개현 8품목\n원유 : 1개현,./tables/001/일본산+수입축산물+방사능검사+현황(2019.8.20)-page-1-table-2.csv
1,1.0,후쿠시마,"원유, 쇠고기, 멧돼지고기, 흰뺨검둥오리고기, 꿩고기, 곰고기, (산)토끼고기, 코퍼\n긴꼬리꿩고기",./tables/001/일본산+수입축산물+방사능검사+현황(2019.8.20)-page-1-table-2.csv
2,2.0,이바라키,멧돼지고기,./tables/001/일본산+수입축산물+방사능검사+현황(2019.8.20)-page-1-table-2.csv
3,3.0,미야기,"쇠고기, 멧돼지고기, 곰고기, 사슴고기",./tables/001/일본산+수입축산물+방사능검사+현황(2019.8.20)-page-1-table-2.csv
4,4.0,이와테,"쇠고기, 사슴고기, 곰고기, 코퍼긴꼬리꿩고기",./tables/001/일본산+수입축산물+방사능검사+현황(2019.8.20)-page-1-table-2.csv


In [37]:
for k,v in col_2_df_foreign.items():
    print(f'{k} columns, {v.shape[0]} rows')

9 columns, 1102 rows
7 columns, 150 rows
6 columns, 60 rows


In [38]:
col_2_df_foreign[9].head(50)

,0,1,2,3,4,5,6,7,8
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),NaN,NaN,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
1,합계,NaN,NaN,NaN,"2,330",NaN,NaN,NaN,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
2,1,고형차,러시아연방,2022.07.31.,100,55,불검출,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
3,2,능이/건조,중국,(2019.08.01.),"2,230",3,-,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),NaN,NaN,./tables/002/201900813+기타국가+방사능+검출+현황-page-1-table-1.csv
1,합계,NaN,NaN,NaN,429,NaN,NaN,NaN,./tables/002/201900813+기타국가+방사능+검출+현황-page-1-table-1.csv
2,1,능이버섯/건조,중국,2020.10.31.,429,13,-,적합,./tables/002/201900813+기타국가+방사능+검출+현황-page-1-table-1.csv
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),NaN,NaN,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv
1,합계,NaN,NaN,NaN,"5,607",NaN,NaN,NaN,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv
2,1,잼,스웨덴,2020.10.27.,806,1,불검출,적합,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv


In [39]:
df_imported = col_2_df_imported[9].copy()
print(df_imported.shape)
df_imported.head()

(147541, 9)


,0,1,2,3,4,5,6,7,8
0,순번,품목명,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:300Bq/kg),NaN,NaN,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
1,합계,NaN,71건,NaN,"105,413",NaN,NaN,NaN,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
2,1,당류가공품,에히메현,2014-10-09,161,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
3,2,청주,효고현,2014-11-04,114,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
4,3,청주,효고현,2014-09-01,95,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv


In [40]:
df_fish_meat = col_2_df_fish_meat[9].copy()
print(df_fish_meat.shape)
df_fish_meat.head()

(53525, 9)


,0,1,2,3,4,5,6,7,8
0,NaN,NaN,NaN,NaN,NaN,세슘\n(기준:100Bq/Kg),요오드\n(기준:300Bq/Kg),판정,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
1,합계,NaN,-,-,"23,876",-,-,-,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
2,1,활돔(참돔),에히메현,2017-03-17,4672,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
3,2,냉장명태,홋카이도,2017-03-17,1157,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
4,3,냉장명태,홋카이도,2017-03-17,1280,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv


In [41]:
df_foreign = col_2_df_foreign[9].copy()
print(df_foreign.shape)
df_foreign.head()

(1102, 9)


,0,1,2,3,4,5,6,7,8
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),NaN,NaN,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
1,합계,NaN,NaN,NaN,"2,330",NaN,NaN,NaN,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
2,1,고형차,러시아연방,2022.07.31.,100,55,불검출,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
3,2,능이/건조,중국,(2019.08.01.),"2,230",3,-,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
0,NaN,NaN,NaN,NaN,NaN,세슘\n요오드\n판정\n(기준:100Bq/kg)\n(기준:100Bq/kg),NaN,NaN,./tables/002/201900813+기타국가+방사능+검출+현황-page-1-table-1.csv


In [42]:
df_imported.loc[~(df_imported[5].notna() & df_imported[5].str.contains('세슘'))].head()

,0,1,2,3,4,5,6,7,8
1,합계,NaN,71건,NaN,"105,413",NaN,NaN,NaN,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
2,1,당류가공품,에히메현,2014-10-09,161,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
3,2,청주,효고현,2014-11-04,114,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
4,3,청주,효고현,2014-09-01,95,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
5,4,발효식초,도쿄도,2014-11-12,144,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv


In [47]:
def process_df(df):
    df = df.copy()
    idx_not_cesium_5 = ~(df[5].notna() & df[5].str.contains('세슘'))
    idx_not_cesium_6 = ~(df[6].notna() & df[6].str.contains('세슘'))
    idx_not_total = df[0]!='합계'
    idx_not_order = df[0]!='순번'
    idx_not_null = df.notna().all(axis=1)
    df.drop(0, axis=1, inplace=True)
    df.columns = ['품목명', '제조·생산지', '제조일자(유통기한)', '수입중량(kg)', '세슘', '요오드', '판정', '파일']
    return df[idx_not_cesium_5 & idx_not_cesium_6 & idx_not_total & idx_not_order & idx_not_null]

In [49]:
df_processed_imported = process_df(df_imported)
print(df_processed_imported.shape)
df_processed_imported.head(5)

(138943, 8)


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일
2,당류가공품,에히메현,2014-10-09,161,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
3,청주,효고현,2014-11-04,114,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
4,청주,효고현,2014-09-01,95,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
5,발효식초,도쿄도,2014-11-12,144,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
6,청주,효고현,2014-11-13,281,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv


In [50]:
df_processed_fish_meat = process_df(df_fish_meat)
print(df_processed_fish_meat.shape)
df_processed_fish_meat.head()

(43705, 8)


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일
2,활돔(참돔),에히메현,2017-03-17,4672,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
3,냉장명태,홋카이도,2017-03-17,1157,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
4,냉장명태,홋카이도,2017-03-17,1280,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
5,냉장대구,홋카이도,2017-03-17,490,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv
6,냉장명태,홋카이도,2017-03-17,1144,불검출,불검출,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv


In [51]:
df_processed_foreign = process_df(df_foreign)
print(df_processed_foreign.shape)
df_processed_foreign.head()

(509, 8)


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일
2,고형차,러시아연방,2022.07.31.,100,55,불검출,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
3,능이/건조,중국,(2019.08.01.),"2,230",3,-,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
2,능이버섯/건조,중국,2020.10.31.,429,13,-,적합,./tables/002/201900813+기타국가+방사능+검출+현황-page-1-table-1.csv
2,잼,스웨덴,2020.10.27.,806,1,불검출,적합,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv
3,차가버섯/건조,러시아연방,(2019.6.13.),"3,520",18,불검출,적합,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv


In [58]:
# df_processed_imported.to_csv('imported.tsv', sep='\t', index=False)

In [59]:
# df_processed_fish_meat.to_csv('fish_meat.tsv', sep='\t', index=False)

In [60]:
# df_processed_foreign.to_csv('foreign.tsv', sep='\t', index=False)

12 columns는 종합 결과. 필요 없음  
7, 6, 2 컬럼은 9 컬럼의 오류 버젼. 수정 필요

Idea
1. 한 row의 모든 값을 \n으로 조인
2. \n으로 다시 스플릿하면 9컬럼이 나올 것이다

In [ ]:
def get_fixed_df_from(flawed):
    data = []
    for idx, row in flawed.iterrows():
        tmp = row.tolist()
        tmp2 = [str(e) for e in tmp if pd.notna(e)]
        joined = '\n'.join(tmp2)
        # exception handling
        joined = joined.replace('\n(세균성)', '(세균성)')
        if '세슘' not in joined:
            splitted = joined.split('\n')
            data.append(splitted)
    return pd.DataFrame(data=data,)

In [ ]:
def post_process_df(fixed):
    idx_4th_numeric = fixed[4].str.replace(',','').str.replace('.','').str.isdecimal()
    idx_5th_nominal = (fixed[5].isin(['검출','불검출']))
    idx_6th_nominal = (fixed[6].isin(['검출','불검출']))
    return fixed[idx_4th_numeric & idx_5th_nominal & idx_6th_nominal]

In [ ]:
# tmp = get_fixed_df_from(col_2_df_imported[7])
# print(len(tmp))
# tmp

In [ ]:
# tmp2 = post_process_df(tmp)
# print(len(tmp2))
# tmp2